In [13]:
# 種々のdatasetをintagrateします

In [14]:
from datasets import load_dataset
import json
import os
from tqdm import tqdm
import yaml

with open('config.yaml', 'r') as file:
    conf= yaml.safe_load(file)
output_path=conf["output_path"]
max_records=conf["max_records"]
print(conf)

{'output_path': '../../data/integrated_text.jsonl', 'max_records': 50000}


In [15]:
#ここでは例として､日英のwikipediaを読み込んでみます

#TODO: よく考えると､これだとRAMが足りなくなるかも?
dataset_list = [
    load_dataset("hpprc/wikipedia-20240101", split="train").shuffle(), #日本語
    load_dataset("wikipedia", "20220301.en",split="train").shuffle(), #英語
]

In [16]:
#datasetの中身を確認してみます｡
#各recordは辞書形式になっています｡大切なのは､ textです｡
for data in dataset_list[0]:
    break
print(data)

{'id': '935053', 'url': 'https://ja.wikipedia.org/wiki/%E3%83%AC%E3%82%A4%E3%83%B3%E3%83%BB%E3%82%B9%E3%83%86%E3%82%A4%E3%83%AA%E3%83%BC', 'title': 'レイン・ステイリー', 'text': 'レイン・トーマス・ステイリー (Layne Thomas Staley, 1967年8月22日 - 2002年4月5日)、出生名レイン・ラザフォード・ステイリー (Layne Rutherford Staley)はアメリカ合衆国のシアトル出身のアリス・イン・チェインズのリードボーカル。\n\n彼の歌声は「麻薬」と形容されるほどクセのある歌声で、当時のグランジ勢の中でも特別に異彩を放っていた。\n\nワシントン州カークランドで生まれ、両親がコレクションしていた音楽を聞いて育つ。影響を受けた音楽はハード・ロック／ヘヴィ・メタルであり、ブラック・サバス、ディープ・パープル、ジューダス・プリースト、ヴァン・ヘイレン、マーシフル・フェイトなど多数を好んで聞いていた。また、ミニストリーやスキニー・パピーなどインダストリアル・ロックやローズ・オブ・ザ・ニュー・チャーチのようなポスト・パンクも好んでいた。1984年にグラム・メタルバンド「Sleze」を結成し、1986年の前身バンド「Alice N\' Chains」を経て、1987年にアリス・イン・チェインズを結成する。\n\n2002年4月19日、ステイリーの会計士から「2週間以上口座から金が引き出されていない」と連絡を受けた母親が警察を伴って訪れた自宅でステイリーは遺体となって発見された。死因はスピードボールによるオーバードースで、発見時の体重は39kg（身長185cm）であった。\n\n幼少期 \n1967年8月22日にアメリカ合衆国ワシントン州カークランドで生まれる。 自身のミドルネームである「ラザフォード」を嫌っており、その名前で呼ばれる度に腹を立てていた。ティーンエイジャーの時に合法的にミドルネームを「トーマス」に変えたが、これはモトリー・クルーのドラマーであるトミー・リーのファンだったからである。2、3歳の頃にベルビュー市でリズム・バンドに加わり、グループの中で最年少であった。9歳の

In [17]:
# 各datasetをmergeして､一つのjsonlにまとめます｡
# ファイルは output_pathに生成されます
# 各datasetは予めクリーニング, dedupされている必要があります｡
#TODO: BTMのため､ジャンル別にデータを並べ替えたい

if os.path.exists(output_path):
    print(f"jsonl file {output_path} already exists. Overwrite? (y/n)")
    ans=input()

    if ans=="y" or "Y":
        overwrite=True
        print("overwriting files...")
        with open(output_path,"w") as f:
            f.write("")
    else:
        overwrite=False
        print("aborted.")
if overwrite:
    with open(output_path,"a") as f:
        for dataset in dataset_list:
            print(dataset)
            cnt=0
            for data in tqdm(dataset):
                out_text=json.dumps({"text":data["text"]}, ensure_ascii=False)
                #jsonlで書き出し

                f.write(out_text+"\n") 

                cnt+=1
                if cnt>max_records:
                    break

jsonl file ../../data/integrated_text.jsonl already exists. Overwrite? (y/n)
overwriting files...
Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1395760
})


  4%|▎         | 50000/1395760 [00:03<01:39, 13586.82it/s]


Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 6458670
})


  1%|          | 50000/6458670 [00:03<07:20, 14550.16it/s]


In [18]:
dataset

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 6458670
})